## Instalação de Requisitos

In [ ]:
# Lendo o arquivo requisitos.txt
with open('requisitos.txt', 'r') as file:
    requirements = file.readlines()

# Removendo espaços em branco e quebras de linha
requirements = [requirement.strip() for requirement in requirements]

# Instalando as dependências usando pip
!pip3 install -U { ' '.join(requirements) }

## Importando o DataSet e Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

file_path = 'NHANES_age_prediction.csv'
data = pd.read_csv(file_path, header=None)

# Deleta a primeira linha onde está as features
data = data.drop(data.index[0])

# Adiciona os nomes traduzidos ao data_set
names = ['id','grupo_idade','idade','genero','atividade_fisica','imc','glicose_pos_jejum','diabetes','saude_bucal','insulina']
data.columns = names

data.head()

## Analisando o DATASET
1. Dimensões do dataset
2. Quantidade de valores NULOS
3. Valores UNICOS
4. Estatísticas

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
for col in data.columns:
    print(f"\nColumn Name: {col}")
    print(data[col].value_counts())

In [ ]:
# Converta todas as colunas para numéricas
dataDescribe = data.apply(pd.to_numeric, errors='coerce')

# Agora você pode usar describe()
dataDescribe.describe()

## Pré-Processamento do DataSet

In [ ]:
# Removendo colunas desnecessárias
# Info importante: > 65 anos é considerado idoso e abaixo = Adulto
data = data.drop('id', axis=1)
data = data.drop('idade', axis=1)

# Transformando a feature grupo_idade(Nosso objetivo) em binário (ou numérico) [Adulto = 0, Idoso = 1]
data['grupo_idade'] = data['grupo_idade'].replace({'Adult': 0, 'Senior': 1})

# Garantindo que todos os dados estão realmente sendo tratados como numéricos
data = data.apply(pd.to_numeric, errors='coerce')

# Adicionar nome das colunas ao para a variável feature_names
data.feature_names = [data.columns]

# Leitura dataset para verificar
data.head()

In [ ]:
data.head()

Testando o melhor método para identificar outliers
1. método IQR (Interquartile Range)

In [ ]:
# Método IQR para identificar e tratar outliers

# Calcular Q1, Q3 e IQR
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1

# Determinar os limites inferior e superior para identificar outliers
lower_bound = Q1 - 3.0 * IQR
upper_bound = Q3 + 3.0 * IQR

# Identificar outliers
outliers = (data < lower_bound) | (data > upper_bound)

# Tratar os outliers (remover ou substituir)
data_tratada = data[~outliers.any(axis=1)]

# Exibir os outliers encontrados
print("Outliers identificados:")
print(data[outliers.any(axis=1)])

#Exibir os dados tratados (sem outliers)
print("\nDados tratados:")
print(data_tratada)

2. Z-Score method

In [ ]:
# Teste de tratamento de dados com Z-Score
# Calcular o Z-score para cada ponto de dados
z_scores = stats.zscore(data)

# Determinar o limite de corte para identificar outliers (por exemplo, 3 desvios padrão)
# Após testes 5.6 foi o melhor valor encontrado para o threshold devido aos diabetes 1 e 3, que em threshholds menores são excluídos, talvez em um dataset maior seja possível encontrar um valor melhor
threshold = 5.6

# Identificar outliers
outliers = (abs(z_scores) > threshold).any(axis=1)

# Tratar os outliers (remover ou substituir)
data_tratada = data[~outliers]

# Exibir os outliers encontrados
print("Outliers identificados:")
print(data[outliers])

# Exibir os dados tratados (sem outliers)
print("\nDados tratados:")
print(data_tratada)

3. Comandos para testar se é utilizável ou não

In [ ]:
data_tratada.describe()
#mean -> média dos valores na coluna
#std -> desvio padrão dos valores na coluna

In [ ]:
data_tratada.shape

In [ ]:
for col in data_tratada.columns:
    print(f"\nColumn Name: {col}")
    print(data_tratada[col].value_counts())

Baseado nos valores, médios podemos confirmar que a melhor alternativa nesse caso será Z-Index com 3 desvios padrão, Então devemos dividir os valores em teste e treino

In [ ]:
data = data_tratada

# Separando nosso objetivo dos dados explicativos, sendo Y o target e X os dados explicativos
x = data.drop('grupo_idade', axis=1)
y = data['grupo_idade']

# Dividindo os dados em Treino e Teste (30% para teste)
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3)

## Criando Máquina Preditiva
1. Algoritmo "Decision Tree"

In [ ]:
# Algoritmo "Decision Tree"
maquina_preditiva = tree.DecisionTreeClassifier()
# OBS = Decision Tree acertou ~76.1% dos casos

# Treinando com os dados de treino
maquina_preditiva.fit(X_train, Y_train)

2. Algoritmo "Random Forest"

In [ ]:
# Algoritmo "Random Forest"

maquina_preditiva = RandomForestClassifier()
# OBS = Random Forest acertou ~83.4% dos casos (melhor resultado)
# OBS.2 = Com o hiperparametro diferenes o resultado foi de ~85.25% de acerto (e se manteve em maioria dos casos testados)

# Treinando com os dados de treino
maquina_preditiva.fit(X_train, Y_train)

In [ ]:
# Testando a máquina preditiva com dados de teste
Valor_predito = maquina_preditiva.predict(X_test)
print(Valor_predito)

## Avaliação da acurácia

In [ ]:
# Avaliando a acurácia
print (((accuracy_score(Valor_predito, Y_test)))*100)